In [2]:
!pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /private/var/folders/m0/7sv6dmqx4xn0n5c2pgc297dw0000gp/T/pip-req-build-d9bx9iht
  Running command git clone -q https://github.com/LIAAD/yake /private/var/folders/m0/7sv6dmqx4xn0n5c2pgc297dw0000gp/T/pip-req-build-d9bx9iht
  Created wheel for yake: filename=yake-0.4.3-py2.py3-none-any.whl size=67198 sha256=c3dbabf8d32eed75ab6fb164bdf897d61056b9323f64560203fc94a147efffcf
  Stored in directory: /private/var/folders/m0/7sv6dmqx4xn0n5c2pgc297dw0000gp/T/pip-ephem-wheel-cache-ggi1zvnv/wheels/be/35/27/e4ebd54b78c1806ed8b0271ce247fcd91e2bedde35889fbc9b
  Created wheel for segtok: filename=segtok-1.5.9-cp37-none-any.whl size=24852 sha256=a7870082515fef3e145f2f919120aae282bda15973703269e87c17e087d40dc0
  Stored in directory: /Users/abuonomo/Library/Caches/pip/wheels/8b/a3/41/54616e3b92f9a3d1b99fc99955a4089f9e6b1d274e66da250c
Successfully built yake segtok
You should consider upgrading via the 'pip install --upgrade pip' command.


In [41]:
import yake
import RAKE
from spacy.lang.en.stop_words import STOP_WORDS
from yake.highlight import TextHighlighter
from IPython.core.display import HTML

In [33]:
rake_extractor = RAKE.Rake(list(STOP_WORDS))
yake_extractor = yake.KeywordExtractor()

In [53]:
th = TextHighlighter(max_ngram_size=3, highlight_pre="<span class='my_class' style='color:blue' >", highlight_post="</span>")

In [34]:
text = """The Ulysses observations have charted the heliosphere around the minimum in the 11 year cycle of solar activity, revealing the large scale properties of the heliospheric medium and its micro-scale characteristics. This book presents for the first time, a comprehensive review of the results of the Ulysses mission.<BR />The first chapter provides an overview of the region, introducing the heliosphere prior to the Ulysses mission, and the objectives and characteristics of the mission itself. Subsequent chapters discuss in detail specific areas of the heliosphere, including the solar wind, large and small scale features, cosmic rays and energetic particles, and the observations of interstellar gas and cosmic dust. Each of these chapters is written by members of the Ulysses science team, concentrating on their own areas of expertise, and is written in a clear and accessible way.<BR />In this book, the authors aim to summarise our understanding of the heliosphere near solar minimum and to provide the basis for understanding the more complex state of the heliosphere around solar maximum, which will be observed during the next phase of the Ulysses mission. <P />"""

In [35]:
rake_kwds = rake.run(text, minFrequency=1, minCharacters=3)
yake_kwds = kw_extractor.extract_keywords(text)

In [66]:
rake_kwds_lim = [(k, v) for k, v in rake_kwds if v > 1.5]
yake_kwds_lim = [(k, v) for k, v in yake_kwds if v < 0.05]

In [67]:
rake_ht = th.highlight(text, rake_kwds_lim)
yake_ht = th.highlight(text, yake_kwds_lim)

In [68]:
HTML(rake_ht)

In [69]:
HTML(yake_ht)